# Managing Big Data for Connected Devices

## 420-N63-NA

## Kawser Wazed Nafi
 ----------------------------------------------------------------------------------------------------------------------------------
    
## Spark Dataframe

Spark Dataframe is a A distributed collection of data grouped into named columns. A DataFrame is equivalent to a relational table in Spark SQL, and can be created using various functions in SparkSession

So far we've only used RDDs in spark. RDDs holds unstructured data. Every entry in the RDD was an object and the RDD contained many objects of the same type. In most cases when we are analyzing data, we would be dealing with structured data. Data that can be represented as a table with columns and rows. 

That's what dataframes are for. Dataframes uses RDDs behind the scenes but allow us to deal with the data as a table. You can think of it as an excel sheet or SQL table. 

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, asc,desc
ss = SparkSession.builder.master("local[4]").appName("movielens-join").getOrCreate();
sc = ss.sparkContext

The Spark Session is used to create dataframes. Before we used the Spark Context to create RDDs.

When we loaded the files using Spark Context into an RDD, the files were handled as a text file. Because it doesn't matter with RDDs if we have columns in the data, every line was an object in our RDD. 

However, with dataframes it does matter. That's why Spark API provides reading functions of different file types and formats. 

In our case we have csv files. We will load the ratings data into a dataframe like this. We specify `header=True` which will take the first line in the file as header in our dataframe.

In [6]:
dfRatings = ss.read.csv("ratings.csv", header=True)
dfRatings.take(5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



### Exercise 1

Why the header data is true? what will happen if you make it false? Can you explain the dataframe differences for the Header true and false?

### Show
Dataframes still follow the lazy evaluation done with RDDs. Well it is using RDDs, just providing us with a better API to handle the data! 

To show our table we can use `show` function which will display the top 20 rows in a table format. You can modify the `n` parameter to display more rows.


In [7]:
dfRatings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
|     1|     50|   5.0|964982931|
|     1|     70|   3.0|964982400|
|     1|    101|   5.0|964980868|
|     1|    110|   4.0|964982176|
|     1|    151|   5.0|964984041|
|     1|    157|   5.0|964984100|
|     1|    163|   5.0|964983650|
|     1|    216|   5.0|964981208|
|     1|    223|   3.0|964980985|
|     1|    231|   5.0|964981179|
|     1|    235|   4.0|964980908|
|     1|    260|   5.0|964981680|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    333|   5.0|964981179|
|     1|    349|   4.0|964982563|
+------+-------+------+---------+
only showing top 20 rows



Similar to RDDs, when applying a transformation to a dataframe it will return a new dataframe. 

However, with the dataframe API, we deal with the data closer to a SQL table than an RDD. <strong>We don't have map and reduce.</strong>  

### Reading a Column
Reading data from a column can be done using select function. The select function will return a new dataframe. 

In [8]:
dfRatings.select('userId').show(n=5)

+------+
|userId|
+------+
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
|     1|
+------+
only showing top 20 rows



We can pass multiple column values and can retrieve multiple columns together.

In [9]:
#we can pass multiple values
dfRatings.select('userId', 'movieId').show(n=5)

+------+-------+
|userId|movieId|
+------+-------+
|     1|      1|
|     1|      3|
|     1|      6|
|     1|     47|
|     1|     50|
|     1|     70|
|     1|    101|
|     1|    110|
|     1|    151|
|     1|    157|
|     1|    163|
|     1|    216|
|     1|    223|
|     1|    231|
|     1|    235|
|     1|    260|
|     1|    296|
|     1|    316|
|     1|    333|
|     1|    349|
+------+-------+
only showing top 20 rows



### Dataframe Column

The column class allows us to reference a specific column to be used in other functions. Filtering for example.

There are multiple ways to get a reference to a column


In [10]:
#Just by using the column name in the df
dfRatings.movieId

Column<'movieId'>

In [11]:
#similar to a dictionary 
dfRatings['movieId']

Column<'movieId'>

We can even use the `col` function from the `pyspark.sql.functions` that was imported in the first cell of this notebook. 

However, you can notice that in the below example our actual dataframe was not even used. Well it doesn't matter, because we are not using the column as a data holder. It is just an indicator of a column with the name we specify. 

In [12]:
col('movieId')

Column<'movieId'>

### Example 1

In the `select` method, we can pass a column object.

In [13]:
dfRatings.select(dfRatings.movieId).show(n=5)

+-------+
|movieId|
+-------+
|      1|
|      3|
|      6|
|     47|
|     50|
+-------+
only showing top 5 rows



In [14]:
#or
dfRatings.select(dfRatings['movieId']).show(n=5)

+-------+
|movieId|
+-------+
|      1|
|      3|
|      6|
|     47|
|     50|
+-------+
only showing top 5 rows



In [15]:
dfRatings.select(col('movieId')).show(n=2)

+-------+
|movieId|
+-------+
|      1|
|      3|
+-------+
only showing top 2 rows



### Usage of Column

We can use columns with arithmatic or boolean expressions to specify an operation.

#this will return a new column object

col('movieId')+10000

In [16]:
#notice that in the new dataframe, the name of the new column is automatically created
dfRatings.select(col('movieId')+10000).show(n=5)

+-----------------+
|(movieId + 10000)|
+-----------------+
|          10001.0|
|          10003.0|
|          10006.0|
|          10047.0|
|          10050.0|
+-----------------+
only showing top 5 rows



In [17]:
#so we can use column functions on it.
(col('movieId')+10000).alias('MovieIdWithAddition')

Column<'(movieId + 10000) AS MovieIdWithAddition'>

We can merge the previous two operations in One

In [18]:
#using that in the select -> MAGIC
dfRatings.select((col('movieId')+10000).alias('MovieIdWithAddition')).show(n=5)

+-------------------+
|MovieIdWithAddition|
+-------------------+
|            10001.0|
|            10003.0|
|            10006.0|
|            10047.0|
|            10050.0|
+-------------------+
only showing top 5 rows



#### What happens when we apply a boolean expression to a column?? Such as col('rating') == 3
We can use that for filtering


In [19]:
dfRatings.filter(col('rating') == 3).show(n=5)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|     70|   3.0|964982400|
|     1|    223|   3.0|964980985|
|     1|    296|   3.0|964982967|
|     1|    316|   3.0|964982310|
|     1|    423|   3.0|964982363|
+------+-------+------+---------+
only showing top 5 rows



we can also combine conditions by adding boolean operators on the returned columns

### Exercise 2
Can you please tell me what is the filtering operation working with the following operation?

In [20]:
dfRatings.filter( (col('rating') == 2.0) &  ((col('rating') > 2) & (col('rating') < 4) | (col('movieId') == 1))).show(n=5)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|   132|      1|   2.0|1157921785|
|   153|      1|   2.0|1525548642|
|   193|      1|   2.0|1435856890|
|   266|      1|   2.0| 945669542|
|   298|      1|   2.0|1447518257|
+------+-------+------+----------+
only showing top 5 rows



### GroupBy

The groupBy function works similar to the SQL groupBy `GroupedData` object which we can apply aggregiate functions to.

In [22]:
dfRatingsCountPerMovie = dfRatings.groupBy(dfRatings.movieId).count()
dfRatingsCountPerMovie.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    296|  307|
|   1090|   63|
| 115713|   28|
|   3210|   42|
|  88140|   32|
|    829|    9|
|   2088|   18|
|   2294|   45|
|   4821|    5|
|  48738|   20|
|   3959|    8|
|  89864|   19|
|   2136|   14|
|    691|    3|
|   3606|    4|
| 121007|    1|
|   6731|    8|
|  27317|    6|
|  26082|    3|
| 100553|    2|
+-------+-----+
only showing top 20 rows



We can then filter out the movies that have less than 20 ratings, means, less than 20 people have rated this movie. 

In [23]:
dfRatingsCountPerMovie.filter(col('count') > 20).show()

+-------+-----+
|movieId|count|
+-------+-----+
|    296|  307|
|   1090|   63|
| 115713|   28|
|   3210|   42|
|  88140|   32|
|   2294|   45|
|  58559|  149|
|  33004|   44|
|  33615|   40|
|   1372|   42|
|   1394|   58|
|   2393|   33|
|   3826|   39|
|   4975|   44|
|    919|   92|
|   1265|  143|
|   2700|   76|
|   3949|   96|
|    926|   24|
|      7|   54|
+-------+-----+
only showing top 20 rows



### Joining dataframes

Now we want to get all the ratings of the movies that have more than 20 ratings. We can join the original dataframe with the dataframe that has the count of ratings per movie.  

The `join` function is applied to a dataframe and takes the other dataframe that we want to join with and the column that we want to join based on. 

First we will join our original dataframe with the count of every movie. So that the original dataframe will have a new column `count` which represent for that movieId how many ratings it has in total. 

In [24]:
# For join, you have to define the column name which is common in between two dataframes and you have selected to perform the join operation
dfRatingsWithCount = dfRatings.join(dfRatingsCountPerMovie, 'movieId')
dfRatingsWithCount.show()

+-------+------+------+---------+-----+
|movieId|userId|rating|timestamp|count|
+-------+------+------+---------+-----+
|      1|     1|   4.0|964982703|  215|
|      3|     1|   4.0|964981247|   52|
|      6|     1|   4.0|964982224|  102|
|     47|     1|   5.0|964983815|  203|
|     50|     1|   5.0|964982931|  204|
|     70|     1|   3.0|964982400|   55|
|    101|     1|   5.0|964980868|   23|
|    110|     1|   4.0|964982176|  237|
|    151|     1|   5.0|964984041|   44|
|    157|     1|   5.0|964984100|   11|
|    163|     1|   5.0|964983650|   66|
|    216|     1|   5.0|964981208|   49|
|    223|     1|   3.0|964980985|  104|
|    231|     1|   5.0|964981179|  133|
|    235|     1|   4.0|964980908|   70|
|    260|     1|   5.0|964981680|  251|
|    296|     1|   3.0|964982967|  307|
|    316|     1|   3.0|964982310|  140|
|    333|     1|   5.0|964981179|   50|
|    349|     1|   4.0|964982563|  110|
+-------+------+------+---------+-----+
only showing top 20 rows



### OrderBy

Before we continue, let's sort by the movieId just to make sure everything is working as we planned.

In [25]:
dfRatingsWithCount.orderBy(col('movieId')).show()

+-------+------+------+----------+-----+
|movieId|userId|rating| timestamp|count|
+-------+------+------+----------+-----+
|      1|    46|   5.0| 834787906|  215|
|      1|   103|   4.0|1431954238|  215|
|      1|    50|   3.0|1514238116|  215|
|      1|    45|   4.0| 951170182|  215|
|      1|    54|   3.0| 830247330|  215|
|      1|     7|   4.5|1106635946|  215|
|      1|    57|   5.0| 965796031|  215|
|      1|    17|   4.5|1305696483|  215|
|      1|    63|   5.0|1443199669|  215|
|      1|    19|   4.0| 965705637|  215|
|      1|    64|   4.0|1161520134|  215|
|      1|    27|   3.0| 962685262|  215|
|      1|    66|   4.0|1104643957|  215|
|      1|    32|   3.0| 856736119|  215|
|      1|    68|   2.5|1158531426|  215|
|      1|    40|   5.0| 832058959|  215|
|      1|    71|   5.0| 864737933|  215|
|      1|    44|   3.0| 869251860|  215|
|      1|    73|   4.5|1464196374|  215|
|      1|    78|   4.0|1252575124|  215|
+-------+------+------+----------+-----+
only showing top

Remember, this data is not filtered! we have all the ratings we started with from the original data but we added a column which has the count of ratings for that movie. 

Now let's filter the ratings to have only the ratings that are for movies with more than 20 ratings.

In [26]:
dfRatingsFiltered = dfRatingsWithCount.filter(col('count') > 20)
dfRatingsFiltered.show()

+-------+------+------+---------+-----+
|movieId|userId|rating|timestamp|count|
+-------+------+------+---------+-----+
|      1|     1|   4.0|964982703|  215|
|      3|     1|   4.0|964981247|   52|
|      6|     1|   4.0|964982224|  102|
|     47|     1|   5.0|964983815|  203|
|     50|     1|   5.0|964982931|  204|
|     70|     1|   3.0|964982400|   55|
|    101|     1|   5.0|964980868|   23|
|    110|     1|   4.0|964982176|  237|
|    151|     1|   5.0|964984041|   44|
|    163|     1|   5.0|964983650|   66|
|    216|     1|   5.0|964981208|   49|
|    223|     1|   3.0|964980985|  104|
|    231|     1|   5.0|964981179|  133|
|    235|     1|   4.0|964980908|   70|
|    260|     1|   5.0|964981680|  251|
|    296|     1|   3.0|964982967|  307|
|    316|     1|   3.0|964982310|  140|
|    333|     1|   5.0|964981179|   50|
|    349|     1|   4.0|964982563|  110|
|    356|     1|   4.0|964980962|  329|
+-------+------+------+---------+-----+
only showing top 20 rows



let's orderBy the count order to find out if the filter is good.

### Exercise 3
Can you spot the differences between GroupBy and OrderBy operation? What are they?

### Exercise 4

Can we define the asending or descending order with orderby or groupby? Can you give a try? Perform the operation in the next cell

### Agg() function

In the `agg` function, we can pass a dictionary. For eache element in the dictionary we specify the column name as key and the aggregation function as value. 

We can also just find the minimum of the count column. We can use the `agg` function. This is usually used with groupBy to specify how do we want to aggregiate the grouped data. But when used on a dataframe ungrouped, then it will treat the full dataframe as one group!



In [27]:
dfRatingsFiltered.agg({'count': 'min'}).show()

+----------+
|min(count)|
+----------+
|        21|
+----------+



In [ ]:
### Sum, Max, Min, Avg

We can also use functions from the `pyspark.sql.functions` and apply them to columns. 

In [28]:
from pyspark.sql.functions import sum, max, min, avg

#we can apply multiple aggregation and each one will become a new column
dfRatingsFiltered.agg(max(col('count')), min(col('count')), 
                      avg(col('count')), min(col('rating')), 
                      max(col('rating')), avg(col('rating'))).show()

+----------+----------+-----------------+-----------+-----------+----------------+
|max(count)|min(count)|       avg(count)|min(rating)|max(rating)|     avg(rating)|
+----------+----------+-----------------+-----------+-----------+----------------+
|       329|        21|84.38338984067929|        0.5|        5.0|3.62677398061748|
+----------+----------+-----------------+-----------+-----------+----------------+



### GroupBy Aggregiate

We want to find the average rating per movie of the filtered ratings we have now. We can groupBy the 'movieId' column and aggregiate to get the average of the rating. 

This is obviously a lot easier than how we did it with RDDs. 

Note how I called alias on the return of the `avg(col('rating'))` because the `avg` function will return a column. 

In [29]:
dfAverageMovieRating = dfRatingsFiltered.groupBy(col('movieId')).agg(avg(col('rating')).alias('avgRating'))
dfAverageMovieRating.show()

+-------+------------------+
|movieId|         avgRating|
+-------+------------------+
|    296| 4.197068403908795|
|   1090| 3.984126984126984|
| 115713|3.9107142857142856|
|   3210|3.4761904761904763|
|  88140|          3.546875|
|   2294|3.2444444444444445|
|  58559| 4.238255033557047|
|  33004|3.4204545454545454|
|  33615|             3.375|
|   1372|3.3452380952380953|
|   1394|3.9913793103448274|
|   2393|3.1515151515151514|
|   3826|2.2948717948717947|
|   4975|3.4204545454545454|
|    919| 3.880434782608696|
|   1265| 3.944055944055944|
|   2700| 3.861842105263158|
|   3949|          3.921875|
|    926| 4.229166666666667|
|      7| 3.185185185185185|
+-------+------------------+
only showing top 20 rows



We can sort by the average. This time we want it desc order! we can call the `desc` function on the column. Which will specify that this column will be in desc order for the orderBy.

In [30]:
dfAverageMovieRating.orderBy(col('avgRating').desc()).show()

+-------+-----------------+
|movieId|        avgRating|
+-------+-----------------+
|    318|4.429022082018927|
|    922|4.333333333333333|
|    898|4.310344827586207|
|    475|              4.3|
|   1204|              4.3|
|    246|4.293103448275862|
|    858|        4.2890625|
|   1235|4.288461538461538|
| 168252|             4.28|
|   2959|4.272935779816514|
|   1276|4.271929824561403|
|    750|4.268041237113402|
|    904|4.261904761904762|
|   1221| 4.25968992248062|
|  48516|4.252336448598131|
|   1213|             4.25|
|   1267|             4.25|
|    912|             4.24|
|  58559|4.238255033557047|
|     50|4.237745098039215|
+-------+-----------------+
only showing top 20 rows



### Finding the name of the movies!

Let's load the movies data into a dataframe and join the movies information with the average rating data. That's simply a join between two dataframes using the movieId column for joining. 

In [33]:
dfMovies = ss.read.csv("movies.csv",header=True)
dfMovies.filter(col('movieId') == 1732).show()
dfMoviesWithAverageRating = dfAverageMovieRating.join(dfMovies, 'movieId')
dfMoviesWithAverageRating.show(n=4)

+-------+--------------------+------------+
|movieId|               title|      genres|
+-------+--------------------+------------+
|   1732|Big Lebowski, The...|Comedy|Crime|
+-------+--------------------+------------+

+-------+------------------+--------------------+--------------------+
|movieId|         avgRating|               title|              genres|
+-------+------------------+--------------------+--------------------+
|    296| 4.197068403908795| Pulp Fiction (1994)|Comedy|Crime|Dram...|
|   1090| 3.984126984126984|      Platoon (1986)|           Drama|War|
| 115713|3.9107142857142856|   Ex Machina (2015)|Drama|Sci-Fi|Thri...|
|   3210|3.4761904761904763|Fast Times at Rid...|Comedy|Drama|Romance|
+-------+------------------+--------------------+--------------------+
only showing top 4 rows



In [34]:
## orderby avgRating

dfMoviesWithAverageRating.orderBy(col('avgRating').desc()).show()

+-------+-----------------+--------------------+--------------------+
|movieId|        avgRating|               title|              genres|
+-------+-----------------+--------------------+--------------------+
|    318|4.429022082018927|Shawshank Redempt...|         Crime|Drama|
|    922|4.333333333333333|Sunset Blvd. (a.k...|Drama|Film-Noir|R...|
|    898|4.310344827586207|Philadelphia Stor...|Comedy|Drama|Romance|
|    475|              4.3|In the Name of th...|               Drama|
|   1204|              4.3|Lawrence of Arabi...| Adventure|Drama|War|
|    246|4.293103448275862|  Hoop Dreams (1994)|         Documentary|
|    858|        4.2890625|Godfather, The (1...|         Crime|Drama|
|   1235|4.288461538461538|Harold and Maude ...|Comedy|Drama|Romance|
| 168252|             4.28|        Logan (2017)|       Action|Sci-Fi|
|   2959|4.272935779816514|   Fight Club (1999)|Action|Crime|Dram...|
|   1276|4.271929824561403|Cool Hand Luke (1...|               Drama|
|    750|4.268041237

### Exercise 5

Can we find the number of ratings per movie with the given example result?  Please perform the logical operation from what you have learned so far.

### Exercise 6
Get to a result similar to the table above but with another column that indicates the total number of ratings each movie has. 


### Exercise 7

In the '6_add_rdd_operation' notebook you were asked to propose 3 things you want to find from the data that uses at least 3 of the files you have. 

Repeat that analysis using dataframes!